In [49]:
# библиотеки
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, udf, row_number
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from pyspark.sql import Row

from geopy.distance import geodesic

In [50]:
# Настройки Spark
conf = SparkConf().setAppName("Bike_Analysis").setMaster("local[*]")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Bike_Analysis, master=local[*]) created by __init__ at <ipython-input-3-2581432d6471>:3 

trips.csv

In [51]:
# Читаем данные
df_trips = spark.read.csv("trips.csv", header=True, inferSchema=True)
# Выводим структуру данных
df_trips.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



stations.csv

In [52]:
# Читаем данные
df_stations = spark.read.csv("stations.csv", header=True, inferSchema=True)
# Выводим структуру данных
df_stations.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)



Найти велосипед с максимальным временем пробега.

In [53]:
# Группируем данные по bike_id и суммируем продолжительность всех поездок каждого велосипеда
bike_time = df_trips.groupBy("bike_id").agg(
    sum("duration").alias("total_duration")  # Создаем новый столбец "total_duration" с суммой времени
)

# Находим велосипед с максимальным временем пробега
max_bike = bike_time.orderBy(col("total_duration").desc()).limit(1)

# Выводим результат: ID велосипеда и его общее время пробега
max_bike.show()

+-------+--------------+
|bike_id|total_duration|
+-------+--------------+
|    535|      18611693|
+-------+--------------+



Найти наибольшее геодезическое расстояние между станциями.


In [54]:
# Функция для вычисления геодезического расстояния (в километрах) между двумя точками
def distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Выбираем id станций и их координаты
stations_list = df_stations.select("id", "lat", "long").collect()

# Инициализируем переменную для максимального расстояния и id станций
max_distance = 0
station_1, station_2 = None, None

# Перебираем все возможные пары станций
for i in range(len(stations_list)):
    for j in range(i + 1, len(stations_list)):  # Исключаем дублирующиеся пары
        dist = distance(
            stations_list[i]["lat"], stations_list[i]["long"],
            stations_list[j]["lat"], stations_list[j]["long"]
        )
        # Обновляем максимальное расстояние и станции, если найдено большее расстояние
        if dist > max_distance:
            max_distance = dist
            station_1 = stations_list[i]["id"]
            station_2 = stations_list[j]["id"]

# Создаем DataFrame для вывода результата
result_df = spark.createDataFrame([
    Row(station_1=station_1, station_2=station_2, distance=max_distance)
])

result_df.show()

+---------+---------+-----------------+
|station_1|station_2|         distance|
+---------+---------+-----------------+
|       16|       60|69.92096757764355|
+---------+---------+-----------------+



Найти путь велосипеда с максимальным временем пробега через станции.

In [55]:
# Получаем ID велосипеда с максимальным временем пробега
max_bike_id = max_bike["bike_id"]

# Фильтруем DataFrame, оставляя только поездки этого велосипеда
bike_path_df = df_trips.filter(col("bike_id") == max_bike_id)

# Выбираем только начальную и конечную станции поездок
bike_path_df = bike_path_df.select("start_station_name", "end_station_name")

# Убираем дубликаты (один и тот же маршрут мог встречаться несколько раз)
bike_path_df = bike_path_df.distinct()

bike_path_df.show(truncate=False)

+---------------------------------------------+---------------------------------------------+
|start_station_name                           |end_station_name                             |
+---------------------------------------------+---------------------------------------------+
|Market at 10th                               |Market at 10th                               |
|Grant Avenue at Columbus Avenue              |Powell at Post (Union Square)                |
|University and Emerson                       |California Ave Caltrain Station              |
|Market at 10th                               |Clay at Battery                              |
|Temporary Transbay Terminal (Howard at Beale)|Harry Bridges Plaza (Ferry Building)         |
|2nd at Folsom                                |San Francisco Caltrain (Townsend at 4th)     |
|Golden Gate at Polk                          |Steuart at Market                            |
|Evelyn Park and Ride                         |Evelyn Park a

Найти количество велосипедов в системе.

In [56]:
# Выбираем столбец bike_id
# Используем distinct(), чтобы оставить только уникальные ID велосипедов
bike_count = df_trips.select("bike_id").distinct().count()
print(f"Общее количество велосипедов в системе: {bike_count}")

Общее количество велосипедов в системе: 700


Найти пользователей потративших на поездки более 3 часов.

In [58]:
# Группируем по zip_code и суммируем продолжительность поездок
users_over_3_hours = df_trips.groupBy("zip_code").agg(
    sum("duration").alias("total_duration")
)

# Фильтруем пользователей, у которых общее время поездок больше 3 часов (10800 секунд)
users_over_3_hours = users_over_3_hours.where(col("total_duration") > 10800)

users_over_3_hours.select("zip_code").show()

+--------+
|zip_code|
+--------+
|   94102|
|   95134|
|   84606|
|   80305|
|   60070|
|   95519|
|   43085|
|   91910|
|   77339|
|   48063|
|   85022|
|    1090|
|    2136|
|   11722|
|   95138|
|   94610|
|   94404|
|   80301|
|   91326|
|   90742|
+--------+
only showing top 20 rows

